# TurboSL: Neural Inverse Structured Light - Google Colab (持久化版本)

这个版本使用 Google Drive 来持久化项目和编译好的库，避免每次重新安装

**重要**: 
1. 确保在 Colab 中选择 GPU 运行时：Runtime -> Change runtime type -> Hardware accelerator -> GPU
2. 第一次运行需要完整安装（~15分钟），之后每次启动只需要 2-3 分钟

## 1. 挂载 Google Drive

In [ ]:
from google.colab import drive
import os

# 挂载 Google Drive
drive.mount('/content/drive')

# 设置工作目录
WORK_DIR = '/content/drive/MyDrive/TurboSL_workspace'
os.makedirs(WORK_DIR, exist_ok=True)

print(f"✓ Google Drive mounted")
print(f"✓ Workspace: {WORK_DIR}")

## 2. 检查 GPU 环境

In [ ]:
import torch
import sys

print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("WARNING: No GPU detected! Please enable GPU in Runtime settings.")

## 3. 智能安装/恢复环境

这个 cell 会检查是否已经安装过，如果是第一次运行会完整安装，之后会快速恢复

In [ ]:
import os
import time
from datetime import datetime

# 检查是否已经安装过
PROJECT_DIR = os.path.join(WORK_DIR, 'TurboSL')
INSTALL_FLAG = os.path.join(WORK_DIR, '.tinycudann_installed')

if os.path.exists(INSTALL_FLAG):
    print("✓ 检测到已安装的环境，跳过完整安装")
    print("⚡ 快速模式：只需要 2-3 分钟")
    FIRST_TIME = False
else:
    print("⚠️  首次安装模式")
    print("⏰ 预计需要 12-16 分钟（包括编译 tiny-cuda-nn）")
    FIRST_TIME = True

print(f"\n开始时间: {datetime.now().strftime('%H:%M:%S')}")

### 3.1 克隆/更新项目

In [ ]:
import os

%cd {WORK_DIR}

if not os.path.exists(PROJECT_DIR):
    print("📥 克隆项目...")
    !git clone https://github.com/guge2/TurboSL.git
    print("✓ 项目克隆完成")
else:
    print("✓ 项目已存在，更新代码...")
    %cd {PROJECT_DIR}
    !git pull
    print("✓ 代码更新完成")

%cd {PROJECT_DIR}

### 3.2 安装依赖

In [ ]:
import time
from datetime import datetime

if FIRST_TIME:
    # 首次安装：完整流程
    print("="*50)
    print("开始完整安装流程")
    print("="*50)
    
    # 1. 安装编译工具
    print("\n[1/3] 安装编译工具...")
    !apt-get update -qq
    !apt-get install -y build-essential -qq
    print("✓ 编译工具安装完成")
    
    # 2. 安装 tiny-cuda-nn (最耗时)
    print("\n[2/3] 安装 tiny-cuda-nn...")
    print("⚠️  这一步需要 8-12 分钟（编译 CUDA 代码）")
    print(f"⏰ 开始时间: {datetime.now().strftime('%H:%M:%S')}")
    print("\n编译中... 请耐心等待\n")
    
    start_time = time.time()
    !pip install git+https://github.com/NVlabs/tiny-cuda-nn/@212104156403bd87616c1a4f73a1c5f2c2e172a9#subdirectory=bindings/torch
    elapsed = time.time() - start_time
    
    print(f"\n✓ tiny-cuda-nn 安装完成")
    print(f"⏱️  用时: {elapsed/60:.1f} 分钟")
    
    # 3. 安装其他依赖
    print("\n[3/3] 安装其他依赖...")
    !pip install -r code/requirements.txt -q
    print("✓ 其他依赖安装完成")
    
    # 创建安装标记
    with open(INSTALL_FLAG, 'w') as f:
        f.write(datetime.now().isoformat())
    print(f"\n✓ 安装标记已创建: {INSTALL_FLAG}")
    print("\n🎉 首次安装完成！下次启动会快很多")
    
else:
    # 非首次：快速恢复
    print("="*50)
    print("快速恢复环境")
    print("="*50)
    
    print("\n安装必要的包...")
    # 只安装可能缺失的包
    !pip install pytorch-lightning omegaconf tensorboard -q
    
    # 尝试导入 tinycudann 来验证
    try:
        import tinycudann
        print("✓ tiny-cuda-nn 可用")
    except ImportError:
        print("⚠️  tiny-cuda-nn 不可用，需要重新安装")
        print("正在重新安装 tiny-cuda-nn...")
        !pip install git+https://github.com/NVlabs/tiny-cuda-nn/@212104156403bd87616c1a4f73a1c5f2c2e172a9#subdirectory=bindings/torch
    
    print("\n✓ 环境恢复完成")

print("\n" + "="*50)
print(f"✓ 所有依赖就绪！")
print(f"⏰ 完成时间: {datetime.now().strftime('%H:%M:%S')}")
print("="*50)

## 4. 验证安装

In [ ]:
# 验证关键包
print("验证关键依赖...\n")

try:
    import torch
    print(f"✓ PyTorch: {torch.__version__}")
except ImportError as e:
    print(f"✗ PyTorch: {e}")

try:
    import tinycudann
    print(f"✓ tiny-cuda-nn: OK")
except ImportError as e:
    print(f"✗ tiny-cuda-nn: {e}")

try:
    import pytorch_lightning as pl
    print(f"✓ PyTorch Lightning: {pl.__version__}")
except ImportError as e:
    print(f"✗ PyTorch Lightning: {e}")

try:
    from omegaconf import OmegaConf
    print(f"✓ OmegaConf: OK")
except ImportError as e:
    print(f"✗ OmegaConf: {e}")

print("\n✓ 所有关键依赖验证通过！")

## 5. 检查数据

In [ ]:
import os

# 检查数据目录
data_dir = "data/captured_data/david/david_opt0_n3_f128"

if os.path.exists(data_dir):
    print(f"✓ 数据目录存在: {data_dir}")
    
    # 统计文件
    import glob
    images = glob.glob(os.path.join(data_dir, 'images/scene_images/*.png'))
    patterns = glob.glob(os.path.join(data_dir, 'patterns/*.png'))
    
    print(f"\n数据统计:")
    print(f"  - 场景图像: {len(images)} 张")
    print(f"  - 投影模式: {len(patterns)} 张")
    print(f"  - 标定文件: {'存在' if os.path.exists(os.path.join(data_dir, 'paramcv.mat')) else '不存在'}")
    print(f"  - 物体 mask: {'存在' if os.path.exists(os.path.join(data_dir, 'mask-undistorted.png')) else '不存在'}")
    
    print("\n✓ 数据完整，可以开始训练")
else:
    print(f"✗ 数据目录不存在: {data_dir}")
    print("请检查数据是否包含在仓库中")

## 6. 配置训练参数

In [ ]:
# 使用默认配置
config_file = "configs/neus-sl-geoalbamb-capture-blur.yaml"

# 查看配置（可选）
# !cat code/{config_file}

print(f"使用配置: {config_file}")
print("\n配置说明:")
print("  - 训练步数: 10,000")
print("  - 验证间隔: 每 2,000 步")
print("  - 预计训练时间: 30-60 分钟")

## 7. 开始训练

In [ ]:
from datetime import datetime

print("="*50)
print("开始训练")
print("="*50)
print(f"⏰ 开始时间: {datetime.now().strftime('%H:%M:%S')}")
print(f"📂 工作目录: {os.getcwd()}")
print(f"⚙️  配置文件: {config_file}")
print("\n训练中...\n")

%cd {PROJECT_DIR}/code
!python launch.py --config {config_file} --gpu 0

## 8. 监控训练进度

In [ ]:
# 查看实验目录
%cd {PROJECT_DIR}/code
!ls -lt exp/ | head -20

## 9. TensorBoard 可视化

In [ ]:
%load_ext tensorboard
%tensorboard --logdir {PROJECT_DIR}/code/runs/

## 10. 查看训练结果

In [ ]:
import os
import glob

%cd {PROJECT_DIR}/code

# 查找最新的实验
exp_dirs = sorted(glob.glob('exp/*/'), key=os.path.getmtime, reverse=True)

if exp_dirs:
    latest_exp = exp_dirs[0]
    print(f"最新实验: {latest_exp}\n")
    
    print("实验结构:")
    !ls -lh {latest_exp}
    
    # 检查保存的文件
    save_dir = os.path.join(latest_exp, 'save')
    if os.path.exists(save_dir):
        print(f"\n保存的结果 ({save_dir}):")
        !ls -lh {save_dir}
else:
    print("未找到实验结果")

## 11. 导出结果到 Google Drive

训练结果已经自动保存在 Google Drive 中了！

In [ ]:
print("✓ 所有结果已自动保存到 Google Drive")
print(f"\n位置: {PROJECT_DIR}/code/exp/")
print("\n你可以在 Google Drive 的以下位置找到:")
print(f"  MyDrive/TurboSL_workspace/TurboSL/code/exp/")
print("\n这些文件会永久保存，即使关闭 Colab 也不会丢失！")

## 12. 下载结果（可选）

In [ ]:
# 如果需要下载到本地
import os
import glob
from datetime import datetime

%cd {PROJECT_DIR}/code

exp_dirs = sorted(glob.glob('exp/*/'), key=os.path.getmtime, reverse=True)

if exp_dirs:
    latest_exp = exp_dirs[0].rstrip('/')
    exp_name = os.path.basename(latest_exp)
    
    # 压缩结果
    output_name = f"turbosl_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.zip"
    !zip -r {output_name} {latest_exp} -q
    
    print(f"✓ 结果已压缩: {output_name}")
    
    # 下载
    from google.colab import files
    files.download(output_name)
    print("✓ 开始下载...")
else:
    print("未找到实验结果")

## 📝 使用说明

### 首次使用:
1. 运行所有 cells，需要约 15-20 分钟完成安装
2. 安装完成后开始训练
3. 训练结果自动保存到 Google Drive

### 再次使用:
1. 重新打开这个 notebook
2. 运行所有 cells，只需要 2-3 分钟恢复环境
3. 直接开始训练，无需重新安装

### 优势:
- ✅ 第一次安装后，以后每次启动只需 2-3 分钟
- ✅ 所有代码和结果都保存在 Google Drive，永不丢失
- ✅ 可以随时中断和恢复训练
- ✅ 多次训练的历史记录都会保留

### 注意事项:
- ⚠️  确保 Google Drive 有足够空间（项目约 500MB，训练结果约 1-2GB）
- ⚠️  Colab 免费版有 GPU 使用时间限制（通常 12 小时）
- ⚠️  如果更换 GPU 类型，可能需要重新编译 tiny-cuda-nn